In [1]:
import openml
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
import itertools
from toolbox import *
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

nodes_combination = [20, 100, 180, 260, 340, 400]
dataset_indices_max = 59
max_shape_to_run = 10000
alpha_range_nn = [0.001, 0.01, 0.1]
subsample = [0.5, 0.8, 1.0]

In [8]:
# Data improted from https://github.com/LeoGrin/tabular-benchmark
# curl -L -o analyses/results/benchmark_total.csv https://figshare.com/ndownloader/files/40081681

X_data_list = []
y_data_list = []
dataset_name = []
def import_datasets():
    SUITE_ID = [334]
    for i in SUITE_ID:
        benchmark_suite = openml.study.get_suite(i)
        for task_id in benchmark_suite.tasks:  # iterate over all tasks
            task = openml.tasks.get_task(task_id)  # download the OpenML task
            dataset = task.get_dataset()
            X, y, categorical_indicator, attribute_names = dataset.get_data(
                dataset_format="dataframe", target=dataset.default_target_attribute
            )   
            X_data_list.append(X)
            y_data_list.append(y)
            dataset_name.append(dataset.name)

            # print(" ")
            # print(" SUITE_ID:", i)
            # print("X_data_list length:", len(X_data_list))
            # print(" ")
    
import_datasets()


In [9]:
print("X_data_list length:", len(X_data_list))
print("y_data_list length:", len(y_data_list))
print("dataset_name length:", len(dataset_name))
unique_dataset_name = set(dataset_name)
print("unique_dataset_name length:", len(unique_dataset_name))
print("dataset_name:", dataset_name)
print("\n")
# print(dataset_name[1])
# print(X_data_list[1].shape)
# # print(y_data_list[4].unique())
# print(X_data_list[1].columns)
# print(X_data_list[1].head(5))
# print("\n")
# print(dataset_name[-7])
# print(X_data_list[-7].shape)
# # print(y_data_list[-4].unique())
# print(X_data_list[-7].columns)
# print(X_data_list[-7].head(5))
# # print(X_data_list[0].head(5))
# for i in range(len(X_data_list)):
#     print(dataset_name[i])
#     print(X_data_list[i].shape)
#     # print(X_data_list[i].columns)
#     # print(X_data_list[i].head(5))
#     print("\n")

X_data_list length: 7
y_data_list length: 7
dataset_name length: 7
unique_dataset_name length: 7
dataset_name: ['electricity', 'eye_movements', 'covertype', 'albert', 'default-of-credit-card-clients', 'road-safety', 'compas-two-years']




In [53]:
from collections import Counter

dataset_name_counts = Counter(dataset_name)

duplicate_dataset_names = [name for name, count in dataset_name_counts.items() if count > 1]

duplicate_datasets = {name: [] for name in duplicate_dataset_names}

for i, name in enumerate(dataset_name):
    if name in duplicate_datasets:
        duplicate_datasets[name].append(i)

for name, indices in duplicate_datasets.items():
    print(f"{name}: {indices}")
    if X_data_list[indices[0]].shape[0] ==  X_data_list[indices[1]].shape[0]:
        print("Same rows")
    else:
        print("Different rows")
        print(X_data_list[indices[0]].shape)
        print(X_data_list[indices[1]].shape)

# print(X_data_list[0].shape)
# print(X_data_list[44].shape)

electricity: [0, 44]
Same rows
eye_movements: [1, 52]
Same rows
covertype: [2, 45]
Different rows
(423680, 54)
(566602, 10)
default-of-credit-card-clients: [4, 55]
Same rows
diamonds: [9, 31]
Same rows
Brazilian_houses: [11, 32]
Same rows
Bike_Sharing_Demand: [12, 33]
Same rows
nyc-taxi-green-dec-2016: [13, 34]
Same rows
house_sales: [14, 35]
Same rows
abalone: [18, 41]
Same rows
delays_zurich_transport: [20, 42]
Same rows
medical_charges: [23, 37]
Same rows
pol: [25, 46]
Different rows
(15000, 26)
(10082, 26)
house_16H: [30, 47]
Different rows
(22784, 16)
(13488, 16)


In [41]:
# def sample_large_datasets(X_data, y_data, max_size=10000):
#     inds = np.random.choice(X_data.index, size=max_size, replace=False)
#     return X_data.iloc[inds], y_data.iloc[inds]

dataset_indices = list(range(dataset_indices_max))
dict_data_indices = {dataset_ind: {} for dataset_ind in dataset_indices}
encode_cnt = 0

for dataset_index, dataset in enumerate(dataset_indices):
    print("\n\nCurrent Dataset: ", dataset)

    X = X_data_list[dataset]
    y = y_data_list[dataset]

    if X.shape[0] > max_shape_to_run:
        X, y = sample_large_datasets(X, y)

    np.random.seed(dataset_index)
    dict_data_indices = find_indices_train_val_test(
        X.shape[0], dict_data_indices=dict_data_indices, dataset_ind=dataset_index
    )
    train_indices = dict_data_indices[dataset_index]["train"]
    val_indices = dict_data_indices[dataset_index]["val"]

    if isinstance(X, np.ndarray):
        X = pd.DataFrame(X)
    # print(X.dtypes)
    
    ### Convert categories data to numerical data
    print("X shape: ", X.shape)
    categorical_columns = X.select_dtypes(include=['object']).columns
    numeric_columns = X.select_dtypes(include=['number']).columns

    encoder = OneHotEncoder(sparse_output=False)
    if len(categorical_columns) > 0:
        X_encoded_strings = encoder.fit_transform(X[categorical_columns])

        X = np.hstack((X[numeric_columns].values, X_encoded_strings))
        print("Encoded", len(categorical_columns), " columns")
        encode_cnt += 1
    else:
        print("No string columns to encode")
    
    print("X_encoded shape: ", X.shape)
    X = pd.DataFrame(X)
    scaler = StandardScaler()
    scaler.fit(X)
    X = scaler.transform(X)
    print("X scalered")
    # break

print("encode_cnt: ", encode_cnt)





Current Dataset:  0
X shape:  (10000, 8)
No string columns to encode
X_encoded shape:  (10000, 8)
X scalered
encode_cnt:  0
